In [13]:
import json
import os

import random 
import math
import numpy as np

import time
from datetime import datetime, timedelta

from PIL import Image
import xml.etree.ElementTree as et

import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
import torchvision.transforms.functional as FT
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

import argparse
import sgrvinod

from sgrvinod import train
from sgrvinod import evaluate

import utils
import transforms
import model_train

In [14]:
parent_directory = "/home/jovyan/work/Test"
path_to_predefined_classes =  "/home/jovyan/work/AST/object_detection/predefined_classes.txt"
batch_size = 16
pretrained = True
keep_difficult =True,
scheduler_name='exponentiallr'
optimizer_name='SGD'
lr=0.005
momentum=0.9
milestones=5
lr_gamma = 0.1
weight_decay=5e-4 
num_epochs = 5
num_workers=2
parent_dir = "/home/jovyan/work/Test"
img_dir = 'chips_positive'
anno_dir = 'chips_positive_xml'
path_to_predefined_classes = "/home/jovyan/work/AST/object_detection/predefined_classes.txt"
subset_img_ids = "test_img_id.txt"
subset_name = "test"
val_size = 0.95

In [15]:
device = torch.device("cpu")

print(device)
# Data loading code
print("Loading data")
train_images, train_objects, val_images, val_objects, test_images, test_objects = utils.split_method(parent_directory, "simple_val",  val_size = val_size)

train_dataset = utils.pascal_voc_dataset(train_images, train_objects, train = True)
val_dataset = utils.pascal_voc_dataset(val_images, val_objects, train = True) #temp set to true to test
test_dataset = utils.pascal_voc_dataset(test_images, test_objects, train = False)

# Custom dataloaders
print("Creating data loaders")
train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle=True, 
                                                num_workers=num_workers, collate_fn = train_dataset.collate_fn, pin_memory=True)
val_data_loader = torch.utils.data.DataLoader(val_dataset,  batch_size = batch_size, shuffle=True, 
                                              num_workers=num_workers, collate_fn = val_dataset.collate_fn, pin_memory=True)
test_data_loader = torch.utils.data.DataLoader(test_dataset,  batch_size = batch_size, shuffle=True, 
                                              num_workers=num_workers, collate_fn = test_dataset.collate_fn, pin_memory=True)

cpu
Loading data
Creating data loaders


In [17]:
print("Creating model")
# Model parameters
label_map = utils.get_label_map(parent_directory, path_to_predefined_classes)
label__color_map = utils.get_label_color_map(label_map)
# replace the classifier with a new one, that has the num_classes which is user-defined
num_classes= len(label_map)  # number of different types of objects

# get number of input features for the classifier
model = sgrvinod.train.get_frcnn_model(num_classes, pretrained)
model.to(device)

## Define make_optimizer() and make_scheduler()
optimizer = sgrvinod.train.make_optimizer(optimizer_name, model, lr=lr, momentum=momentum, weight_decay=weight_decay)
lr_scheduler = sgrvinod.train.make_scheduler(scheduler_name, optimizer, milestones=[milestones], lr_gamma = lr_gamma)

# Calculate total number of epochs to train and the epochs to decay learning rate at (i.e. convert iterations to epochs)
# To convert iterations to epochs, divide iterations by the number of iterations per epoch
# The paper trains for 120,000 iterations with a batch size of 32, decays after 80,000 and 100,000 iterations

    
for epoch in range(num_epochs):
    # Decay learning rate at particular epochs
            
    # train for one epoch, printing every 10 iterations
    sgrvinod.train.train(train_loader=train_data_loader,
                   model=model,
                   optimizer=optimizer,
                   epoch=epoch,
                   device=device,
                   print_freq = 10)

    # update the learning rate
    lr_scheduler.step()
    
    # evaluate on the test dataset
    sgrvinod.evaluate.evaluate(val_data_loader, model.eval(), device = device)

    # Save checkpoint
    sgrvinod.train.save_checkpoint(epoch, model, optimizer)

Creating model
Epoch: [0][15/13]	Batch Time 151.441 (151.441)	Data Time 0.994 (0.994)	Loss 2.9003 (2.9003)	
Epoch: [0][15/13]	Batch Time 144.474 (145.917)	Data Time 0.000 (0.091)	Loss 0.5024 (0.9804)	


Evaluating:   0%|          | 0/243 [00:21<?, ?it/s]


ValueError: too many values to unpack (expected 2)

In [18]:
# Make sure it's in eval mode

model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d()
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d()
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d()
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d()
          (relu): ReLU(inplace=True)
          (downsample): Sequent

In [21]:
# Lists to store detected and true boxes, labels, scores
det_boxes = list()
det_labels = list()
det_scores = list()
true_boxes = list()
true_labels = list()
true_difficulties = list()  # it is necessary to know which objects are 'difficult', see 'calculate_mAP' in utils.py

In [ ]:
from tqdm import tqdm
with torch.no_grad():
    # Batches
    for i, (images, boxes, labels, difficulties) in enumerate(tqdm(val_data_loader, desc='Evaluating')):
        images = images.to(device)  # (N, 3, 300, 300)
        
        # Forward prop.
        predicted_boxes, predicted_labels, predicted_scores = model(images)

Model Evaluate

In [ ]:
# Parameters
data_folder = './'
keep_difficult = True  # difficult ground truth objects must always be considered in mAP calculation, because these objects DO exist!
batch_size = 64
workers = 4
device = torch.device("cpu")
checkpoint = './checkpoint_ssd300.pth.tar'

# Load model checkpoint that is to be evaluated
checkpoint = torch.load(checkpoint)
model = checkpoint['model']
model = model.to(device)

# Switch to eval mode
model.eval()